导入库

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import time
import datetime
from sklearn.model_selection import train_test_split

import cv2

import h5py
%matplotlib inline

random_state = 424
dataset_path = 'dataset'

这个数据集是26位司机的各种样子的照片。所以我按照司机来划分训练集和验证集。

In [3]:
# 读取csv文件
dataset = pd.read_csv('dataset/driver_imgs_list.csv')

# 随机选择两个司机作为测试集
driver_id = dataset['subject'].unique()
driver_id_test = np.random.choice(driver_id, 2)
print(driver_id_test)

# 组合路径
dataset['path']=dataset[['classname', 'img']].apply(lambda x: '/'.join(x), axis=1)

# 将路径转换成list格式
train_path = dataset['path'].loc[~dataset['subject'].isin(driver_id_test)].tolist()
test_path = dataset['path'].loc[dataset['subject'].isin(driver_id_test)].tolist()

['p002' 'p047']


In [4]:
import os
import shutil


if os.path.exists('dataset/train2'):
    print('split dataset exist')
else:
    os.mkdir('dataset/train2')
    os.mkdir('dataset/valid2')
    for i in range(10):
        os.mkdir('dataset/train2/c'+str(i))
        os.mkdir('dataset/valid2/c'+str(i))
    # 复制图片到新的数据集
    if os.path.exists('dataset/train2'):
        print('train dataset copy begain')
        for filename in tqdm(train_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/train2/'+filename)
    if os.path.exists('dataset/valid2'):
        print('test dataset copy begain')
        for filename in tqdm(test_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/valid2/'+filename)




  0%|          | 1/20864 [00:00<1:06:08,  5.26it/s]

train dataset copy begain


  5%|▌         | 82/1560 [00:00<00:01, 818.93it/s]

test dataset copy begain


100%|██████████| 1560/1560 [00:01<00:00, 925.78it/s]


预训练模型模板

In [2]:
from keras.models import Sequential, Model
from keras.applications import *
from keras.layers import Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.layers.normalization import *
from keras.optimizers import *
from keras.preprocessing.image import *
from keras.callbacks import ModelCheckpoint

train_dir = 'dataset/train2'  # 训练集数据
val_dir = 'dataset/valid2' # 验证集数据
nb_classes = len(glob.glob(train_dir + "/*"))  # 分类数


batch_size = 128
epochs = 5
# def VGG16_model(X_train, y_train, X_test, y_test):
def run_a_model(BASE_MODEL,input_shape, fine_tune_layer, preprocessing=None, model_name_option=''):
    input_height = input_shape[0]
    input_width = input_shape[1]
    input_tensor = Input((input_height, input_width, 3))
    base_model = BASE_MODEL(input_tensor=Input((input_height, input_width, 3)),
                            weights='imagenet', 
                            include_top=False, 
                            input_shape=(input_height, input_width, 3)
                           )
    
    
    x = input_tensor
    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, x)    
    print("total layer count {}".format(len(base_model.layers)))
    
    for i in range(fine_tune_layer):
        model.layers[i].trainable = False
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.1,
        zoom_range=0.1,
        preprocessing_function=preprocessing
    )
    
    test_gen = ImageDataGenerator(
        preprocessing_function=preprocessing
    )
    
    train_generator = train_gen.flow_from_directory(train_dir, (input_height, input_width), 
                                                    shuffle=True, batch_size=batch_size, class_mode='categorical')
    test_generator = test_gen.flow_from_directory(val_dir, (input_height, input_width), 
                                                  shuffle=True, batch_size=batch_size, class_mode='categorical')
    
    
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_valid_sample = test_generator.samples // batch_size + 1
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit_generator(
        train_generator,
        steps_per_epoch=steps_train_sample,
        epochs=5,
        validation_data=test_generator,
        validation_steps=steps_valid_sample)
    
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        steps_per_epoch=steps_train_sample,
        epochs=5,
        validation_data=test_generator,
        validation_steps=steps_valid_sample)    
    
    model.save("models/model_{}{}.h5".format(BASE_MODEL.func_name, model_name_option))

    return model


Using TensorFlow backend.


In [ ]:
def show_loss_plot(model):
    # list all data in history
    print(model.history.keys())
    # summarize history for loss
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

单模型预测

In [ ]:
# VGG16 15 fine tune layer 
vgg16 = run_a_model(VGG16, (224, 224), 15, model_name_option='_BN_15_SGD_10epoche')

In [ ]:
# ResNet50 155 finetune layer with BN 10 epoches
resnet50 = run_a_model(ResNet50, (224, 224), 155, model_name_option='_BN_155_SGD_10epoche')

In [ ]:
# InceptionV3 200 finetune layer with BN 10 epoches
inceptionv3 = run_a_model(InceptionV3, (299, 299), 200, 
                          preprocessing=inception_v3.preprocess_input, model_name_option='_BN_200_SGD_10epoche')

下面的舍弃

In [11]:
# ResNet50 154 finetune layer with BN 10 epoches
run_a_model(ResNet50, (224, 224), 160, preprocessing=None, model_name_option='_BN_160')

total layer count 175
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 282s - loss: 0.3123 - acc: 0.9097 - val_loss: 1.7951 - val_acc: 0.4924
Epoch 2/5
163/163 [==============================] - 262s - loss: 0.0667 - acc: 0.9799 - val_loss: 1.6361 - val_acc: 0.6151
Epoch 3/5
163/163 [==============================] - 262s - loss: 0.0466 - acc: 0.9845 - val_loss: 2.0096 - val_acc: 0.5748
Epoch 4/5
163/163 [==============================] - 262s - loss: 0.0321 - acc: 0.9895 - val_loss: 1.3769 - val_acc: 0.6811
Epoch 5/5
163/163 [==============================] - 262s - loss: 0.0226 - acc: 0.9918 - val_loss: 1.4909 - val_acc: 0.6946
Epoch 1/5
163/163 [==============================] - 277s - loss: 0.0151 - acc: 0.9948 - val_loss: 1.2312 - val_acc: 0.7141
Epoch 2/5
163/163 [==============================] - 262s - loss: 0.0103 - acc: 0.9964 - val_loss: 1.1701 - val_acc: 0.7147
Epoch 3/5
163/163 [====

In [19]:
# vgg16 with BN,finetune layer 15  10 epoche
run_a_model(VGG16, (224, 224), 15, preprocessing=None, model_name_option='_BN_15')

total layer count 19
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 369s - loss: 0.2424 - acc: 0.9305 - val_loss: 3.5156 - val_acc: 0.4374
Epoch 2/5
163/163 [==============================] - 343s - loss: 0.0398 - acc: 0.9882 - val_loss: 2.2552 - val_acc: 0.5803
Epoch 3/5
163/163 [==============================] - 344s - loss: 0.0192 - acc: 0.9952 - val_loss: 2.3463 - val_acc: 0.6897
Epoch 4/5
163/163 [==============================] - 344s - loss: 0.0132 - acc: 0.9962 - val_loss: 2.1964 - val_acc: 0.5791
Epoch 5/5
163/163 [==============================] - 344s - loss: 0.0108 - acc: 0.9972 - val_loss: 1.0301 - val_acc: 0.7703
Epoch 1/5
163/163 [==============================] - 347s - loss: 0.0076 - acc: 0.9985 - val_loss: 1.0063 - val_acc: 0.7385
Epoch 2/5
163/163 [==============================] - 343s - loss: 0.0034 - acc: 0.9992 - val_loss: 1.0395 - val_acc: 0.7367
Epoch 3/5
163/163 [=====

In [20]:
run_a_model(InceptionV3, (299, 299), 180, preprocessing=inception_v3.preprocess_input, model_name_option='_BN_180')

total layer count 311
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 506s - loss: 0.2324 - acc: 0.9389 - val_loss: 0.6160 - val_acc: 0.8082
Epoch 2/5
163/163 [==============================] - 489s - loss: 0.0385 - acc: 0.9894 - val_loss: 0.8902 - val_acc: 0.7666
Epoch 3/5
163/163 [==============================] - 489s - loss: 0.0316 - acc: 0.9911 - val_loss: 0.8109 - val_acc: 0.8302
Epoch 4/5
163/163 [==============================] - 489s - loss: 0.0157 - acc: 0.9951 - val_loss: 0.6184 - val_acc: 0.8552
Epoch 5/5
163/163 [==============================] - 489s - loss: 0.0148 - acc: 0.9956 - val_loss: 1.3184 - val_acc: 0.7166
Epoch 1/5
163/163 [==============================] - 497s - loss: 0.0075 - acc: 0.9975 - val_loss: 0.9058 - val_acc: 0.7954
Epoch 2/5
163/163 [==============================] - 489s - loss: 0.0037 - acc: 0.9989 - val_loss: 0.8289 - val_acc: 0.8216
Epoch 3/5
163/163 [====

In [21]:
run_a_model(InceptionV3, (299, 299), 200, preprocessing=inception_v3.preprocess_input, model_name_option='_BN_200')

total layer count 311
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 472s - loss: 0.2829 - acc: 0.9258 - val_loss: 0.8754 - val_acc: 0.7502
Epoch 2/5
163/163 [==============================] - 464s - loss: 0.0513 - acc: 0.9857 - val_loss: 0.5692 - val_acc: 0.8180
Epoch 3/5
163/163 [==============================] - 464s - loss: 0.0228 - acc: 0.9929 - val_loss: 1.3623 - val_acc: 0.7434
Epoch 4/5
163/163 [==============================] - 464s - loss: 0.0163 - acc: 0.9950 - val_loss: 0.8817 - val_acc: 0.8088
Epoch 5/5
163/163 [==============================] - 464s - loss: 0.0156 - acc: 0.9952 - val_loss: 0.3770 - val_acc: 0.9023
Epoch 1/5
163/163 [==============================] - 473s - loss: 0.0094 - acc: 0.9968 - val_loss: 0.4018 - val_acc: 0.8821
Epoch 2/5
163/163 [==============================] - 464s - loss: 0.0043 - acc: 0.9987 - val_loss: 0.5527 - val_acc: 0.8503
Epoch 3/5
163/163 [====

优秀模型继续优化

In [18]:
# VGG16 5 fine tune layer 5 more epoche
run_a_model(VGG16, (224, 224), 15, model_name_option='_BN_15_20epoche')

total layer count 19
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/10
163/163 [==============================] - 341s - loss: 0.2312 - acc: 0.9318 - val_loss: 6.6858 - val_acc: 0.3800
Epoch 2/10
163/163 [==============================] - 341s - loss: 0.0453 - acc: 0.9873 - val_loss: 3.3952 - val_acc: 0.5987
Epoch 3/10
163/163 [==============================] - 341s - loss: 0.0191 - acc: 0.9943 - val_loss: 3.3759 - val_acc: 0.6109
Epoch 4/10
163/163 [==============================] - 341s - loss: 0.0149 - acc: 0.9953 - val_loss: 3.0096 - val_acc: 0.5589
Epoch 5/10
163/163 [==============================] - 341s - loss: 0.0128 - acc: 0.9964 - val_loss: 2.6761 - val_acc: 0.5993
Epoch 6/10
163/163 [==============================] - 341s - loss: 0.0092 - acc: 0.9969 - val_loss: 3.4047 - val_acc: 0.4918
Epoch 7/10
163/163 [==============================] - 341s - loss: 0.0117 - acc: 0.9962 - val_loss: 3.7892 - val_acc: 0.5492
Epoch 8/10
163/16

In [6]:
run_a_model(VGG16, (224, 224), 15, model_name_option='_BN_15_SGD')

total layer count 19
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
Epoch 1/5
164/164 [==============================] - 385s - loss: 0.2480 - acc: 0.9271 - val_loss: 1.7392 - val_acc: 0.5641
Epoch 2/5
164/164 [==============================] - 341s - loss: 0.0405 - acc: 0.9893 - val_loss: 2.3495 - val_acc: 0.6333
Epoch 3/5
164/164 [==============================] - 341s - loss: 0.0246 - acc: 0.9930 - val_loss: 1.7060 - val_acc: 0.6218
Epoch 4/5
164/164 [==============================] - 341s - loss: 0.0138 - acc: 0.9964 - val_loss: 1.2859 - val_acc: 0.7218
Epoch 5/5
164/164 [==============================] - 341s - loss: 0.0073 - acc: 0.9977 - val_loss: 2.6110 - val_acc: 0.5994
Epoch 1/5
164/164 [==============================] - 342s - loss: 0.0079 - acc: 0.9974 - val_loss: 0.4301 - val_acc: 0.8827
Epoch 2/5
164/164 [==============================] - 340s - loss: 0.0078 - acc: 0.9978 - val_loss: 0.2986 - val_acc: 0.9282
Epoch 3/5
164/164 [=====

In [4]:
run_a_model(VGG16, (224, 224), 15, model_name_option='_BN_15_SGD_20e')

total layer count 19
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
Epoch 1/5
164/164 [==============================] - 603s - loss: 0.2486 - acc: 0.9283 - val_loss: 2.4089 - val_acc: 0.5801
Epoch 2/5
164/164 [==============================] - 337s - loss: 0.0393 - acc: 0.9889 - val_loss: 1.0648 - val_acc: 0.6955
Epoch 3/5
164/164 [==============================] - 337s - loss: 0.0212 - acc: 0.9943 - val_loss: 1.8312 - val_acc: 0.5647
Epoch 4/5
164/164 [==============================] - 337s - loss: 0.0141 - acc: 0.9958 - val_loss: 1.6120 - val_acc: 0.6782
Epoch 5/5
164/164 [==============================] - 337s - loss: 0.0099 - acc: 0.9969 - val_loss: 1.5405 - val_acc: 0.7071
Epoch 1/5
164/164 [==============================] - 338s - loss: 0.0067 - acc: 0.9982 - val_loss: 0.4048 - val_acc: 0.8929
Epoch 2/5
164/164 [==============================] - 336s - loss: 0.0057 - acc: 0.9987 - val_loss: 0.3907 - val_acc: 0.8942
Epoch 3/5
164/164 [=====

In [13]:
run_a_model(ResNet50, (224, 224), 160, model_name_option='_BN_160_SGD')

total layer count 175
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
Epoch 1/5
164/164 [==============================] - 283s - loss: 0.2960 - acc: 0.9125 - val_loss: 1.3964 - val_acc: 0.6301
Epoch 2/5
164/164 [==============================] - 268s - loss: 0.0623 - acc: 0.9800 - val_loss: 1.7165 - val_acc: 0.5949
Epoch 3/5
164/164 [==============================] - 270s - loss: 0.0395 - acc: 0.9875 - val_loss: 1.8411 - val_acc: 0.6558
Epoch 4/5
164/164 [==============================] - 270s - loss: 0.0365 - acc: 0.9892 - val_loss: 1.5533 - val_acc: 0.6891
Epoch 5/5
164/164 [==============================] - 269s - loss: 0.0328 - acc: 0.9897 - val_loss: 0.9221 - val_acc: 0.7506
Epoch 1/5
164/164 [==============================] - 283s - loss: 0.0212 - acc: 0.9933 - val_loss: 0.9448 - val_acc: 0.7359
Epoch 2/5
164/164 [==============================] - 269s - loss: 0.0197 - acc: 0.9935 - val_loss: 0.9816 - val_acc: 0.7346
Epoch 3/5
164/164 [====

In [5]:
from keras.models import load_model
model = load_model('models/model_ResNet50_BN_160_SGD.h5')
y_predictions, test_id = make_predictions(model, (224, 224), 128)
create_submission(y_predictions, test_id, tag="Resnet_160_SGD")

Found 79726 images belonging to 1 classes.
623/623 [==============================] - 859s   


In [ ]:
run_a_model(InceptionV3, (299, 299), 200, preprocessing=inception_v3.preprocess_input, model_name_option='_BN_200_SGD')
# finish

In [9]:
model = load_model('models/model_InceptionV3_BN_200_SGD.h5')
y_predictions, test_id = make_predictions(model, (299, 299), 128, preprocessing=inception_v3.preprocess_input)
create_submission(y_predictions, test_id, tag="Inception_200_SGD")

Found 79726 images belonging to 1 classes.
623/623 [==============================] - 1283s  


In [10]:
run_a_model(InceptionV3, (299, 299), 170, preprocessing=inception_v3.preprocess_input, model_name_option='_BN_170_SGD')
model = load_model('models/model_InceptionV3_BN_170_SGD.h5')
y_predictions, test_id = make_predictions(model, (299, 299), 128, preprocessing=inception_v3.preprocess_input)
create_submission(y_predictions, test_id, tag="Inception_170_SGD")

total layer count 311
Found 20864 images belonging to 10 classes.
Found 1560 images belonging to 10 classes.
Epoch 1/5
134/164 [=======================>......] - ETA: 89s - loss: 0.2937 - acc: 0.9253

KeyboardInterrupt: 

In [ ]:
run_a_model(Xception, (299, 299), 135, preprocessing=xception.preprocess_input, model_name_option='_BN_135_SGD')

In [ ]:
run_a_model(InceptionV3, (299, 299), 180, preprocessing=inception_v3.preprocess_input, model_name_option='_BN_180_SGD')

基础预测

In [7]:
from keras.models import load_model
model = load_model('models/model_VGG16_BN_15_SGD_20e.h5')

In [4]:
def get_img(path, image_shape):
    img = cv2.imread(path, 0)
    resized = cv2.resize(img, image_shape)
    return resized

def KNN(input_data, predictions, N=5, coef = [0.05, 0.1, 0.15, 0.2, 0.5]):
    new_predictions_list = []
    for i in tqdm(range(len(input_data))):
        distance = np.square(np.subtract(input_data, input_data[i]))
        distance = np.sqrt(np.sum(distance, axis=(1, 2)))
        N_min_index = np.argsort(distance)[::-1][-N:]
        
        pred = [ predictions[N_min_index[j]] * coef[j] for j in range(N)]
        pred = np.sum(pred, axis=0)
        new_predictions_list.append(pred)
                
    return np.array(new_predictions_list)

def make_predictions(MODEL, image_size, batch_size, preprocessing=None):
    gen = ImageDataGenerator(preprocessing_function=preprocessing)
    path_test_data = 'dataset/to_prediction'
    test_generator = gen.flow_from_directory(path_test_data,  image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode='categorical')
    y_predictions = MODEL.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    # y_predictions = y_predictions.clip(min=0.005, max=0.995)
    
    test_id = list()
    for i, file_name in enumerate(test_generator.filenames):
        flbase = os.path.basename(file_name)
        test_id.append(flbase)       
    
    return y_predictions, test_id

def create_submission(predictions, test_id, tag = ""):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + tag + '.csv')
    result1.to_csv(sub_file, index=False)

In [9]:
y_predictions, test_id = make_predictions(model, (224, 224), 128)

Found 79726 images belonging to 1 classes.
623/623 [==============================] - 1756s  


In [22]:
new_predictions = y_predictions.clip(min=0.005, max=0.995)

In [10]:
create_submission(y_predictions, test_id, tag="VGG16_SGD_20e")